# **Working with RDD (Resilient Distributed Dataset)**

**`Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark`**

**`Author: Amin Karami (PhD, FHEA)`**

---

**Resilient Distributed Dataset (RDD)**: RDD is the fundamental data structure of Spark. It is fault-tolerant (resilient) and immutable distributed collections of any type of objects.

source: https://spark.apache.org/docs/latest/rdd-programming-guide.html

source: https://spark.apache.org/docs/latest/api/python/reference/

In [1]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

In [2]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
# !pip3 install -q findspark
# import findspark
# findspark.init()
########## ONLY in Ubuntu Machine ##########

In [3]:
# Linking with Spark
from pyspark import SparkContext, SparkConf

In [4]:
# Initializing Spark
conf = SparkConf().setAppName("RDD_practice").setMaster("local[*]")
sc = SparkContext(conf=conf)
print(sc)

<SparkContext master=local[*] appName=RDD_practice>


In [5]:
sc.defaultParallelism

2

# **Part 1: Create RDDs and Basic Operations**
# **There are two ways to create RDDs:**

1.   Parallelizing an existing collection in your driver program
2.   Referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

In [6]:
# Generate random data:
import random
randomlist = random.sample(range(0, 40), 10)
print(randomlist)

[25, 32, 21, 7, 0, 39, 30, 20, 35, 10]


In [7]:
# Create RDD:
rdd1 = sc.parallelize(randomlist, 4)
rdd1.collect()

[25, 32, 21, 7, 0, 39, 30, 20, 35, 10]

In [8]:
# Data distribution in partitions:
print(rdd1.getNumPartitions())  # it is an action
print(rdd1.glom().collect())
print("the two partitions", rdd1.glom().take(2))

4
[[25, 32], [21, 7], [0, 39], [30, 20, 35, 10]]
the two partitions [[25, 32], [21, 7]]


In [9]:
# Print last partition
print(rdd1.glom().collect()[3])

[30, 20, 35, 10]


In [10]:
# count(): it is an action
rdd1.count()

10

In [11]:
# first(): it is an action
rdd1.first()

25

In [12]:
# top(): it is an action
rdd1.top(2)
# ordena de forma ascendente y luego toma los ultimos 2 valores

[39, 35]

**Las acciones no se hacen en paralelo, tener cuidado, por ejemplo, la accion top necesita tomar los registros (todos en caso de no existir transformaciones, como takeSample, takeOrdered), ordenarlos y luego hacer el top, esto podría estar operando sobre todos los datos si no se tiene cuidado**.

**Las Transformaciones como map, filter, distinct son las que si se ejecutan en paralelo**.

Una buena forma de diferenciar acciones de transformaciones es pensar en que las transformaciones devuleven un nuevo rdd y al hacer print sobre la salida de estas funciones obtienes objetos no legibles por personas

In [13]:
# distinct(): it is a transformation
rdd1.distinct().collect()

[32, 0, 20, 25, 21, 30, 10, 7, 39, 35]

In [14]:
# map(): it is a transformation
rdd_map = rdd1.map(lambda item: (item + 1)*3)
rdd_map.glom().collect()

[[78, 99], [66, 24], [3, 120], [93, 63, 108, 33]]

In [15]:
# filter(): it is a transformation
rdd_filter = rdd1.filter(lambda x: x%3==0)
rdd_filter.glom().collect()

[[], [21], [0, 39], [30]]

In [16]:
# flatMap(): it is a transformation
"""
la salida es una lista unidimensional aunque la transformacion no lo sea
suele hacerse para aplicar funciones de agregacion las cuales deben ser
conmutativa y asociativas para que funcione correctamente,
recuerda que los datos se particionan y se van tomando de diferentes sitios en
memoria o particiones para aplicar la funcion definida"""
rdd_flatmap = rdd1.flatMap(lambda x: [x+2, x+3])
print(rdd_flatmap.glom().collect())
rdd_flatmap.reduce(lambda x, y: x+y)

[[27, 28, 34, 35], [23, 24, 9, 10], [2, 3, 41, 42], [32, 33, 22, 23, 37, 38, 12, 13]]


488

In [17]:
# Descriptive statistics: they are actions
print([rdd1.max(), rdd1.min(), rdd1.mean(), round(rdd1.stdev(), 2), rdd1.sum()])

[39, 0, 21.9, np.float64(12.2), 219]


In [18]:
# mapPartitions(): it's a transformation
"""
Retorna lista con cantidad de particiones, cada elemento es el resultado de
aplicar el calculo definido en el argumento (funcion) en cada particion"""
def myfunc(partition):
  sum = 0
  for item in partition:
    sum += item
  """
  yield va acumulando el resultado hasta que terminen todos los llamados a esta
  funcion, si usamos un return, en cada llamado a la función intentará retornar
  el valor y como resultado saltará un error
  """
  yield sum

rdd1.mapPartitions(myfunc).collect()

[57, 28, 39, 95]

# **Part 2: Advanced RDD Transformations and Actions**

In [27]:
# union(): it is a transformation
print(rdd1.glom().collect())
rdd2 = sc.parallelize([1, 14, 20, 28, 10, 13, 3], 2)
print(rdd2.glom().collect())

rdd_union = rdd1.union(rdd2)
print(rdd_union.glom().collect())

rdd_union.getNumPartitions()

"""
la funcion crea un nuevo RDD con todas particiones sumadas de cada RDD
individual, por lo tanto incrementa la cantidad de particiones
"""

[[25, 32], [21, 7], [0, 39], [30, 20, 35, 10]]
[[1, 14, 20], [28, 10, 13, 3]]
[[25, 32], [21, 7], [0, 39], [30, 20, 35, 10], [1, 14, 20], [28, 10, 13, 3]]


'\nla funcion crea un nuevo RDD con todas particiones sumadas de cada RDD\nindividual, por lo tanto incrementa la cantidad de particiones\n'

In [31]:
# intersection(): it is a transformation
rdd_intersection = rdd1.intersection(rdd2)
print(rdd_intersection.collect())
"""Es como un inner join. La cantidad de particiones es la suma de las
particiones de cada RDD individial. Es posible que queden particiones vacías"""
print(rdd_intersection.getNumPartitions())
rdd_intersection.glom().collect()

[20, 10]
6


[[], [], [20], [], [10], []]

In [35]:
# Find empty partitions
counter = 0
for item in rdd_intersection.glom().collect():
  counter += 1 if len(item)==0 else 0
print(counter)

4


In [36]:
# coalesce(numPartitions):
"""para reducir cantidad de particiones, se recomienda para quitar particiones
vacías. También se puede hacer con repartition(numPartitions) pero es menos
eficiente"""
rdd_intersection.coalesce(1).glom().collect()

[[20, 10]]

In [49]:
# takeSample(withReplacement, num, [seed])
"""a diferencia de take, takesample toma valores aleatorios"""
rdd1.takeSample(False, 5, 2025)

[7, 39, 30, 21, 25]

In [50]:
# takeOrdered(n, [ordering]). It is an action
# ascending ordering is default
# puedes pasar una funcion para ordernarlo a gusto
print(rdd1.takeOrdered(3))
rdd1.takeOrdered(3, key=lambda x: -x)

[0, 7, 10]


[39, 35, 32]

In [53]:
# reduce(): It's an action
# para aplicar funciones de agregacion
print(rdd1.reduce(lambda x, y: x+y))
print(rdd1.sum())

219
219


In [70]:
# reduceByKey(): It's a transformation
"""Similiar a reduce pero además de ser una transformacion, se aplica sobre
RDD's que sean de tipo clave, valor (son como listas de tuplas de 2 valores
cada tupla, por ejemplo [('a', 1), ('b', 1), ('c', 3)]), si se aplica sobre
otros tipos de RDD (por ejemplo una lista simple de valores), arroja un error.
Dicho de otra manera, esto es como agrupar un dataframe y aplicar funciones
de agregación a cada grupo (key)"""
rdd_rbk = sc.parallelize(
    [('a', 1), ('b', 2), ('c', 3), ('a', 4), ('b', 5), ('c', 6)]
    , 2
)
print(rdd_rbk.glom().collect())
rdd_rbk.reduceByKey(lambda x,y: x*y).glom().collect()

[[('a', 1), ('b', 2), ('c', 3)], [('a', 4), ('b', 5), ('c', 6)]]


[[('b', 10), ('c', 18)], [('a', 4)]]

In [71]:
# sortByKey():  # It is a transformation
# para ordenar por las keys
rdd_rbk.reduceByKey(lambda x,y: x*y).sortByKey().glom().collect()

[[('a', 4), ('b', 10)], [('c', 18)]]

In [62]:
# countByKey(). It is an action
# para contar elementos por key
print(rdd_rbk.countByKey())
print(rdd_rbk.countByKey().items())  # otra forma de mostrar

defaultdict(<class 'int'>, {'a': 2, 'b': 2, 'c': 2})
dict_items([('a', 2), ('b', 2), ('c', 2)])


In [67]:
# groupByKey(): It is a transformation
"""a pesar de ser una transformacion, no se recomienda utilizar sobre grandes
datasets, todas las combinaciones de clave y valor son enviador al driver para
luego agrupar, similar a lo que hace una accion, MUCHO CUIDADO!.

La funcion retorna una lista de grupos/keys con un objeto iterable"""
rdd_grouped = rdd_rbk.groupByKey()
print(rdd_grouped.getNumPartitions())
print(rdd_grouped.collect())
for item in rdd_grouped.collect():
  print(item[0], [values for values in item[1]])

2
[('b', <pyspark.resultiterable.ResultIterable object at 0x7e4632bf8ad0>), ('c', <pyspark.resultiterable.ResultIterable object at 0x7e46337bb310>), ('a', <pyspark.resultiterable.ResultIterable object at 0x7e4632bfb750>)]
b [1, 5]
c [3, 6]
a [1, 4]


In [72]:
# lookup(key): It's an action
"""devuelve una lista con los valores de todos los items de una clave. Se
aplica sobre RDD's de clave y valor"""
rdd_rbk.lookup('b')

[2, 5]

In [73]:
# cache:
# By default, each transformed RDD may be recomputed each time you run an action on it.
# However, you may also persist an RDD in memory using the persist (or cache) method,
# in which case Spark will keep the elements around on the cluster for much faster access the next time you query it.
rdd_rbk.persist()


ParallelCollectionRDD[199] at readRDDFromFile at PythonRDD.scala:289

In [74]:
# Persistence (https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)

from pyspark import StorageLevel

rdd1.persist(StorageLevel.MEMORY_AND_DISK)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289